### Parallel Fibonacci (Java) [ungraded]

Let us explore dynamic multithreading using a recursive algorithm. The `n`-th Fibonacci number for `n ≥ 0` can be computed by a sequential, recursive procedure:

```algorithm
procedure fib(n: integer) → (r: integer)
    x ← fib(n – 1)
    y ← fib(n – 2)
    r := x + y
```

This algorithm is not a practical way to compute large Fibonacci numbers, as the same computation is repeated. For example, a call to `fib(6)` recursively calls `fib(5)` and `fib(4)`. But the call to `fib(5)` also calls `fib(4)`. Both instances of `fib(4)` return the same result, `fib(4) = 3`. Without *memoization*, the second call to `fib(4)` replicates the work that the first call performs.

Although procedure `fib` is a poor way to compute Fibonacci numbers, it makes a good example for key concepts in the design of multithreaded algorithms. Observe that within `fib(n)`, the two recursive calls to `fib(n – 1)` and `fib(n – 2)` are independent of each other: they could be called in either order, and the computation performed by one in no way affects the other. Therefore, the two recursive calls can run in parallel. Implement `parallelfib` accordingly!

In [11]:
%%writefile Fib.java    
public class Fib {

    static class Fibb extends Thread{
        int n;
        int r;

        public Fibb(int n){
            this.n = n;
        }

        public void run(){
            this.r = parallelfib(this.n);
        }
    }
    
    public static int parallelfib(int n){
        if (n <= 1) return n;
        else{

            Fibb x = new Fibb(n-1);
            Fibb y = new Fibb(n-2);

            x.start(); y.start();
            try{x.join(); y.join();} catch (Exception e) {}

            return x.r + y.r;
            
        }
    }
    
    public static void main(String args[]) {
        int n = Integer.parseInt(args[0]);

        final long start = System.currentTimeMillis();
        int r = parallelfib(n);
        final long end = System.currentTimeMillis();
        
        System.out.println(r + " in " + (end - start) + " ms");
    }
}

Overwriting Fib.java


In [12]:
!javac Fib.java

In [13]:
!java Fib 10

55 in 53 ms


In [14]:
!java Fib 15

610 in 536 ms


In [15]:
!java Fib 20

6765 in 17741 ms


### Limiting the Number of Threads

The overhead of thread creation and scheduling may slow down a program if many threads are created. To address this for `parallelfib`, we add another parameter, `p`, for the maximal number of threads to be created. When `p` is `1`, the computation continues sequentially. In `parallelfib`, each of the threads computing `fib(n – 1)` and `fib(n – 2)` continues to create (approximately) `p / 2` threads. Complete the template below!

In [20]:
%%writefile FibP.java

class WorkerP extends Thread{
    int n; int p; int r;
    public WorkerP(int n, int p){this.n = n; this.p = p;}
    public void run(){
        this.r = FibP.parallelfib(this.n, this.p);
    }
}

public class FibP {
    static int sequentialfib(int n) {
        if (n <= 1) return n;
        else {
            int x = sequentialfib(n - 1);
            int y = sequentialfib(n - 2);
            return x + y;
        }
    }
    static int parallelfib(int n, int p) {
        if (p <= 1) return sequentialfib(n);
        else if (n <= 1) return n;
        else {
            WorkerP x = new WorkerP(n-1, p/2);
            WorkerP y = new WorkerP(n-2, p/2);
            x.start(); y.start();
            try{x.join(); y.join();} catch (Exception e) {}
            return x.r + y.r;
        }
    }
    public static void main(String args[]) {
        int n = Integer.parseInt(args[0]);
        int p = Integer.parseInt(args[1]);

        long start = System.currentTimeMillis();
        int r = parallelfib(n, p);
        long end = System.currentTimeMillis();
        System.out.println("Parallel: " + r + " by " + (end - start) + " ms");

        start = System.currentTimeMillis();
        r = sequentialfib(n);
        end = System.currentTimeMillis();
        System.out.println("Sequential: " + r + " by " + (end - start) + " ms");
    }
}

Overwriting FibP.java


In [21]:
!javac FibP.java

In [22]:
!java FibP 20 4

Parallel: 6765 by 3 ms
Sequential: 6765 by 0 ms


In [23]:
!java FibP 30 4

Parallel: 832040 by 5 ms
Sequential: 832040 by 4 ms


In [24]:
!java FibP 40 1

Parallel: 102334155 by 520 ms
Sequential: 102334155 by 552 ms


In [25]:
!java FibP 40 2

Parallel: 102334155 by 339 ms
Sequential: 102334155 by 585 ms


In [26]:
!java FibP 40 4

Parallel: 102334155 by 228 ms
Sequential: 102334155 by 539 ms


In [27]:
!java FibP 40 8

Parallel: 102334155 by 130 ms
Sequential: 102334155 by 521 ms


In [28]:
!java FibP 40 16

Parallel: 102334155 by 87 ms
Sequential: 102334155 by 511 ms


In [29]:
!java FibP 40 32

Parallel: 102334155 by 61 ms
Sequential: 102334155 by 511 ms


In [30]:
!java FibP 40 256

Parallel: 102334155 by 123 ms
Sequential: 102334155 by 535 ms


In [31]:
!java FibP 40 1024

Parallel: 102334155 by 555 ms
Sequential: 102334155 by 519 ms


What can you observe about the elapsed time with increasing value of `p`? Explain!

Your answer here

*Note.* The time complexity of the parallel matrix multiplication can be analyzed by the *work and depth model*. The *work* `𝒲` is the total time to execute the entire computation by one worker. The *depth* `𝒟` is the longest time to execute the computation by infinitely many workers. In other words, the work is the sum of the times taken by all workers, and the depth is the longest time taken by a worker. The parallel time complexity is `O(𝒲 / 𝒫 + 𝒟)`, where `𝒫` is the total number of workers that can separately perform the total work. Here, each worker corresponds to a processor core, rather than a thread. 

* The work is given by the recurrence `T(n) = T(n – 1) + T(n – 2) + O(1)`. From that, we can get that the work is the same as that of the sequential version, which is `O(ϕⁿ)` where `ϕ = (1 + √5)/2`.
* The depth is given by the recurrence `T(n) = max(T(n – 1), T(n – 2)) + O(1)`, as two asymmetric `fib` sub-procedures  run in parallel. From that, we can get the depth is `O(n)`.
* The total parallel running time is `O(ϕⁿ / P + n)`, where `P` is the number of workers.